In [17]:

using LinearAlgebra
using GenericLinearAlgebra


In [18]:
function complete_orthonormal(U::Matrix{Float64})
    d, r = size(U)
    r ≤ d || error("U deve ter no máximo d colunas")

    r == d && return zeros(Float64, d, 0)   # já completa

    # Projeta vetores aleatórios no complemento de span(U)
    Z = randn(Float64, d, d - r)
    Z .-= U * (U' * Z)                      # remove componente em U

    # QR fino garante ortonormalidade
    C = Matrix(qr(Z).Q)                  # d × (d−r)
    return C
end

complete_orthonormal (generic function with 1 method)

In [19]:
function build_csd_input_matrix(D1::Matrix{Float64}, D2::Matrix{Float64})
    # Dimensões
    m, d = size(D1)  # D1: m x d
    n, _ = size(D2)  # D2: n x d

    # Passo 1: QR de D1^T para obter base de R(D1) em R^d
    Q1 = Matrix(qr(D1').Q)  # Q1 é d x m
    R1 = Matrix(qr(D1').R)  # R1 é m x m

    println("Q1: ", size(Q1))  # Debugging: verificar tamanho de Q1
    println("R1: ", size(R1))  # Debugging: verificar tamanho de R1

    r1 = sum(abs.(diag(R1)) .> eps() * maximum(abs.(diag(R1))))  # Estimar posto
    E1 = Q1[:, 1:r1]  
    E2 = complete_orthonormal(E1)  # Base ortonormal completa para R^d, d x (d - r1)
    E = hcat(E1, E2)  # Base ortonormal completa para R^d, d x d
    
    # Passo 2: QR de D2^T para obter base de R(D2) em R^d
    Q2 = Matrix(qr(D2').Q)  # Q2 é d x n
    R2 = Matrix(qr(D2').R)  # R2 é n x n

    println("Q2: ", size(Q2))  # Debugging: verificar tamanho de Q2
    println("R2: ", size(R2))  # Debugging: verificar tamanho de R2

    c1 = sum(abs.(diag(R2)) .> eps() * maximum(abs.(diag(R2))))  # Estimar posto
    F1 = Q2[:, 1:c1]  
    F2 = complete_orthonormal(F1)  # Base ortonormal completa para R^d, d x (d - c1)
    

    # Passo 3: Calcular Q11 = E1^T F1
    Q11 = E1' * F1  # r1 x c1
    Q12 = E1' * F2  # r1 x (d - c1)
    Q21 = E2' * F1  # (d - r1) x c1
    Q22 = E2' * F2  # (d - r1) x (d - c1)

    # Passo 4: Concatenar Q11, Q12, Q21, Q22
    Qtop = hcat(Q11, Q12)
    Qbot = hcat(Q21, Q22)
    Q = vcat(Qtop, Qbot)
    
    return Q, r1, c1, E, E1, F1, R1, R2
end

build_csd_input_matrix (generic function with 1 method)

In [20]:
# Exemplo com dados aleatórios
n = 100  # amostras dos datasets
d = 500  # número de features

D1 = randn(n, d)  # dataset 1
D2 = randn(n, d)  # dataset 2

Q, r1, c1, E, E1, F1, R1, R2 = build_csd_input_matrix(D1, D2)

Q1: (500, 100)
R1: (100, 100)
Q2: (500, 100)
R2: (100, 100)


([-0.0184711773659502 -0.015619493931736332 … -0.10259075616486744 0.017717869317142365; -0.07589614522262966 0.0798436855843189 … 0.01697555760012345 0.00674901621099592; … ; 0.04963526097687966 0.0037471726955763084 … 0.046796391960739356 -0.01861825912413302; -0.03089833305019686 -0.001739398053617957 … -0.03773719571013553 -0.0167968621041212], 100, 100, [-0.09212292206248196 -0.04307747626498185 … -0.07502367088536903 -0.02815640273866346; -0.0367537526386082 -0.042640005452307506 … 0.013785964817532428 0.0331742783913384; … ; -0.01070797926859714 0.011836147875032453 … 0.029124811498243326 -0.050556529971653455; 0.040879112587656495 -0.011006570719379207 … -0.06726224118882906 0.013837213275629206], [-0.09212292206248196 -0.04307747626498185 … -0.08396768721225424 0.001027783855937348; -0.0367537526386082 -0.042640005452307506 … 0.03500606667981031 0.03430585689947727; … ; -0.01070797926859714 0.011836147875032453 … -0.03671986552555615 -0.025098761556132665; 0.040879112587656495

In [21]:
display(size(Q))  # Verifica o tamanho da matriz Q
println("Posto de D1: ", r1)
println("Posto de D2: ", c1)
println("Tamanho de E: ", size(E))  # Verifica o tamanho da base ortonormal completa E
println("Tamanho de E1: ", size(E1))  # Verifica o tamanho de E1
println("Tamanho de F1: ", size(F1))  # Verifica o tamanho de E2
println("Tamanho de R1: ", size(R1))  # Verifica o tamanho de R1
println("Tamanho de R2: ", size(R2))  # Verifica o tamanho de R2

(500, 500)

Posto de D1: 100
Posto de D2: 100
Tamanho de E: (500, 500)
Tamanho de E1: (500, 100)
Tamanho de F1: (500, 100)
Tamanho de R1: (100, 100)
Tamanho de R2: (100, 100)


In [22]:
# Verificar se R1' * E1' é aproximadamente igual a D1
if R1'*E1' ≈ D1
    println("R1' * E1' é aproximadamente igual a D1")
else
    println("R1' * E1' é diferente de D1")
end

R1' * E1' é aproximadamente igual a D1


In [23]:
function rebuild_datasets_matrices(E, R)
    D_reconstructed = R' * E'  
    # Retornar a matriz reconstruída
    return D_reconstructed
end

rebuild_datasets_matrices (generic function with 1 method)

In [24]:
function verify_Q(Q::Matrix{Float64})
    if Q'*Q ≈ I
        println("A matriz Q é ortonormal")
    else
        println("A matriz Q não é ortonormal")
        
    end
end

verify_Q (generic function with 1 method)

In [25]:
# Exibir e verificar a corretude da matriz A
verify_Q(Q)
D1_reconstructed = rebuild_datasets_matrices(E1, R1)
D2_reconstructed = rebuild_datasets_matrices(F1, R2)

if D1_reconstructed ≈ D1 && D2_reconstructed ≈ D2
    println("As matrizes reconstruídas são equivalentes às originais!")
else
    println("As matrizes reconstruídas NÃO são equivalentes às originais.")
        
end

A matriz Q é ortonormal
As matrizes reconstruídas são equivalentes às originais!


In [26]:
function csd(Q, c1, r1, E)
    d= size(Q, 1)  # Dimensão do espaço
    # separar Q em blocos
    Q11 = Q[1:r1, 1:c1]  # r1 x c1
    Q12 = Q[1:r1, c1+1:end]  # r1 x (d - c1)
    Q21 = Q[r1+1:end, 1:c1]  # ( d - r1) x c1
    Q22 = Q[r1+1:end, c1+1:end]  # (d - r1) x (d - c1)

    # SVD dos blocos de Q = EᵀF
    U1, S11, V1 = svd(Q11; full=true)     
    U2, S22, V2  = svd(Q22; full=true)

    D11 = U1' * Q11 * V1
    D12 = U1' * Q12 * V2 
    D21 = U2' * Q21 * V1
    D22 = U2' * Q22 * V2
    
    # Monta matriz diagonalizada D via blocos projetados
    D_top = hcat(D11, D12)  # r1 x d
    D_bot = hcat(D21, D22)  # (d - r1) x (d - c1)
    D = vcat(D_top, D_bot)  # d × d

    # Constrói matriz U unificada
    U = Matrix{Float64}(I, d, d)
    U[1:r1, 1:r1] = U1
    U[r1+1:end, r1+1:end] = U2
    
    # Constrói matriz V unificada
    V= Matrix{Float64}(I, d, d)
    V[1:c1, 1:c1] = V1
    V[c1+1:end, c1+1:end] = V2
    
    # Calcula os ângulos θ entre os subespaços
    θ = acos.(clamp.(S11, -1.0, 1.0))

    # Reconstrói a rotação W que leva E1 → F1
    W = E * U * D * U' * E'


    return U, U1, U2, V, V1, V2, D, θ, W  
end

csd (generic function with 1 method)

In [27]:
U, U1, U2, V, V1, V2, D, θ, W =csd(Q, c1, r1, E)

([-0.10892249577576943 0.006877655456195342 … 0.0 0.0; 0.08354615926663017 -0.04025893937219999 … 0.0 0.0; … ; 0.0 0.0 … 0.045341755662018465 -0.033962751433661564; 0.0 0.0 … -0.035445597278641626 -0.00732803148640546], [-0.10892249577576943 0.006877655456195342 … 0.08764345642721111 -0.13014737402412993; 0.08354615926663017 -0.04025893937219999 … 0.09126128813903299 0.020480862555881146; … ; -0.10039036225692136 -0.005291996730262543 … -0.03811067812805901 0.1560586903414804; -0.16618395863191252 -0.1389868662582321 … -0.12586360359484186 -0.03041014286283096], [-0.036579663490897515 0.02678258887534313 … -0.09824010474374455 0.03306530268113713; -0.02034627401633079 0.07396640124173852 … 0.010382680820675035 0.04251715492384802; … ; -0.008472533288218986 0.015342126163255428 … 0.045341755662018465 -0.033962751433661564; -0.01383775897346282 0.008056061323512574 … -0.035445597278641626 -0.00732803148640546], [0.08084002115112737 0.10730872166734193 … 0.0 0.0; 0.09412598526416652 -0.02

In [28]:
θ_deg = θ .* (180 / π)
println("Ângulos principais entre subespaços (em graus): ", θ_deg)
println("tamanho de U: ", size(U))
println("tamanho de U1: ", size(U1))
println("tamanho de U2: ", size(U2))
println("tamanho de V: ", size(V))
println("tamanho de V1: ", size(V1))
println("tamanho de V2: ", size(V2))
println("tamanho de D: ", size(D))
println("tamanho de W: ", size(W))

Ângulos principais entre subespaços (em graus): [38.45962810135467, 40.300038675671665, 41.0378023935142, 41.88847006065002, 42.508168534590354, 43.653597996481324, 44.00449017544559, 44.99991888834761, 45.23436821507954, 45.97117876830103, 46.47704943803909, 47.599719748860025, 48.441003324371664, 48.907927050276555, 49.63172309842743, 49.647052569002724, 50.27289140640379, 50.63595963243996, 51.065606633032445, 51.43222764364425, 51.971521347821806, 52.49931967216916, 53.27009858360114, 54.239332874135826, 55.20670776479365, 55.71883905673532, 56.13087058878054, 56.49218566561542, 56.805211565875894, 57.493265991619545, 57.91303565614927, 58.88317511713263, 59.57609735660782, 59.89593721399649, 60.31465605275751, 60.5260208681832, 61.11869490564769, 61.63454544111254, 62.114505010239355, 62.45687984633449, 63.19326242374534, 63.26006846633715, 63.87715279619683, 63.99679766949859, 65.13388204733297, 65.42223159326093, 65.99076291282479, 66.07845750563452, 66.78193428545784, 67.079093

In [29]:
display(θ)                 # Ângulos principais entre subespaços (em radianos)
@show(U*D*V' ≈ Q) 
display(norm(W-I) ≈ norm(D-I))  # Deve ser próximo de 0 (rotação direta correta)
@show W*E1*U1*V1' ≈ F1 

100-element Vector{Float64}:
 0.6712471394611743
 0.7033683635715258
 0.7162447695496325
 0.7310917211814117
 0.7419074999212548
 0.7618990153805991
 0.7680232392230227
 0.7853967477320524
 0.7894886604126086
 0.8023484305297643
 ⋮
 1.5004380652738702
 1.5081380198443575
 1.519620418038426
 1.5304248428576093
 1.5367182238013133
 1.5422489739174035
 1.5495211650200171
 1.5533659619829876
 1.5650853199335986

true

U * D * V' ≈ Q = true
W * E1 * U1 * V1' ≈ F1 = true


true

In [30]:
D2_reconstructed_from_rotation = rebuild_datasets_matrices(W*E1*U1*V1', R2)

if  D2_reconstructed_from_rotation ≈ D2
    println("A matriz D2 reconstruída a partir da rotação é equivalente à original!")
else
    println("A matriz D2 reconstruída a partir da rotação NÃO é equivalente à original.")
end

A matriz D2 reconstruída a partir da rotação é equivalente à original!
